In [1]:
!nltk.download('stopwords')

import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import SnowballStemmer
import pymorphy2
from pymystem3 import Mystem
from string import punctuation

stop_words = stopwords.words('russian')
stop_words.remove("не")
stop_words.remove("ни")
mystem = Mystem()
stemmer = SnowballStemmer(language='russian')
morph = pymorphy2.MorphAnalyzer()

/bin/bash: -c: line 0: syntax error near unexpected token `'stopwords''
/bin/bash: -c: line 0: `nltk.download('stopwords')'


In [2]:

def preprocess(text, is_stem = False, is_morph = True, is_remove_stop_words = True):
    if text is None:
        text = ""
    text = text.lower()
    text = re.sub(r'\[.*?\]|\(.*?\)|\n|,|\.|\xa0|;', "", text).replace(chr(769), "")
    # tokens = word_tokenize(text, language='russian')
    tokens = mystem.lemmatize(text)
    if is_remove_stop_words:
        tokens = [token for token in tokens if token not in stop_words\
            and token != " " and token != '' \
            and token.strip() not in punctuation]

    if is_stem:
        tokens = list(map(lambda x: stemmer.stem(x), tokens))
    if is_morph:
        tokens = list(map(lambda x: morph.parse(x)[0].normal_form, tokens))

    return ' '.join(tokens)

'екатерина i   —  российский императрица 1721 год'

In [ ]:
# text = "Екатери́на I (Ма́рта Самуи́ловна Скавро́нская, в браке Крузе; после принятия православия Екатери́на Алексе́евна Миха́йлова; 5 [15] апреля 1684 — 6 [17] мая 1727, Санкт-Петербург) — российская императрица с 1721 года (как супруга царствующего императора)"
# preprocess(text)

# PREPROCESS FOR LABELING

In [30]:
import pandas as pd
origin_result_data = pd.read_csv("../spacy/data/result.csv")
origin_result_data.head()

,Unnamed: 0,original,scored_text,distance,score
0,0,"Вели́кое кня́жество Влади́мирское, до 1157года...",С середины XIII века сюзеренитет великих княз...,0.923924,5
1,1,Новгоро́дская респу́блика — севернорусское сре...,Находясь с 1245 года под сюзеренитетом велики...,0.908120,5
2,2,Вели́кое кня́жество Лито́вское— восточноевропе...,С 1385 года находилось в личной унии с Королев...,0.861883,5
3,3,Вели́кое кня́жество Моско́вское— средневековое...,Возвышению Москвы и укреплению её авторитета ...,0.901416,5
4,4,Оле́г — князь Новгородский с 879 года и велики...,Получив власть над новгородскими землями посл...,0.916509,5


In [75]:
def preprocess_for_labeling(data):
    after_preprocess = data["original"].transform(preprocess)

    after_preprocess = after_preprocess.dropna()
    after_preprocess = after_preprocess.drop_duplicates()

    with open('/home/droman/Documents/diploma/spacy/data/data_for_labeling_with_preprocess.txt', mode='wt', encoding='utf-8') as myfile:
        myfile.write('\n'.join(set(after_preprocess.tolist())))


def preprocess_for_classification(data):
    for title in ["original", "scored_text"]:
        after_preprocess = data[title].transform(preprocess)
        data[title] = after_preprocess

    data.dropna(how='any', inplace=True)
    data.drop_duplicates()

    data.to_csv('/home/droman/Documents/diploma/deeppavlov_ner_3.6/data/data_with_preprocess.csv', index=False)

In [76]:
preprocess_for_classification(origin_result_data)


In [83]:
result = pd.read_csv("./data/data_with_preprocess.csv")

In [84]:
result.loc[112]

Unnamed: 0                                                   113
original       святослав ярослав крещение николай год     : «...
scored_text                   1054 год ярослав поделить русь сын
distance                                                0.948365
score                                                          3
Name: 112, dtype: object

In [86]:
print(len(result))
# result.dropna(how='any', inplace=True)
# print(len(result))

161


In [80]:
result.loc[112]

Unnamed: 0                                                   112
original       олег      —      князь новгородский 879 год ве...
scored_text                                                  NaN
distance                                                       0
score                                                          3
Name: 112, dtype: object

In [52]:
origin = pd.read_csv("../spacy/data/result.csv")
print(origin.loc[112]["scored_text"])
preprocess(origin.loc[112]["scored_text"])

''

In [49]:
print(len(result))

162
